# Code 2 - SQL Agents

## Referances
- Parsing Errors - https://github.com/langchain-ai/langchain/issues/14947

## Step 1 - Connect to Database

### Import Packages

In [1]:
from sqlite3 import connect
from langchain_community.utilities.sql_database import SQLDatabase

### Create database connection for LLM (create db)

In [2]:
db = SQLDatabase.from_uri("sqlite:///chinook.db", sample_rows_in_table_info = 3)
print(len(db.get_usable_table_names()), db.get_usable_table_names())

11 ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


### Check Connection

In [3]:
con = connect("chinook.db")
cur = con.execute("Select Count(Distinct(AlbumId)) from Album;")
print(cur.fetchall())

[(347,)]


## Step 2 - Setup Langchain SQL Chain

### Import API Key

In [4]:
#import os
#from dotenv import load_dotenv
#load_dotenv()

In [5]:
#API_KEY = os.getenv("OPENAI_API_KEY")

### Import Packages

In [6]:
from langchain_ollama.llms import OllamaLLM

### Create an Instance of LLM with configuration & Check (creating llm)

In [7]:
llm = OllamaLLM(model="llama3")
llm

OllamaLLM(model='llama3')

## Step 3 Setup Agent

Advantages of SQL Agents:

1. Saves tokens - Retrieving only required schema & relevant tables.
2. Recover from errors by generating queries, catch the traceback and regenerate correctly
3. Chains run on a predefined pattern; Agents is defined by llm; The LLM then chooses the right tools to execute

### Import Packages

In [8]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType

### Create an Instance of Agent with llm and db

In [9]:
agent_executor = create_sql_agent(
    llm, 
    db=db, 
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True, 
    stream_runnable=False,
    agent_executor_kwargs=dict(handle_parsing_errors=True)
)

### Execute LLM Agent

In [10]:
result = agent_executor.invoke("Which country's customers spent the most?")
print(result)



> Entering new SQL Agent Executor chain...
Let's get started.

Action: sql_db_list_tables
Action Input: (empty stringAlbum, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThought: Now that I have the list of tables, I need to think about which ones are relevant to this question. It seems like "Customer" and maybe "Invoice" could be important. Let me check the schema of those tables.

Action: sql_db_schema
Action Input: Customer, Invoice
CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from C

### Check Execution Flow

In [19]:
print(agent_executor)

name='SQL Agent Executor' verbose=True agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessage(content='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.\nYou can order the results by a relevant colu

In [20]:
print(agent_executor.agent.runnable.get_prompts()[0].messages[0].content)

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't k

In [21]:
agent_executor.agent.runnable.to_json()['kwargs']['middle'][1].kwargs['tools']

[{'type': 'function',
  'function': {'name': 'sql_db_query',
   'description': "Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.",
   'parameters': {'type': 'object',
    'properties': {'query': {'description': 'A detailed and correct SQL query.',
      'type': 'string'}},
    'required': ['query']}}},
 {'type': 'function',
  'function': {'name': 'sql_db_schema',
   'description': 'Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3',
   'parameters': {'type': 'object',
    'properties': {'table_names': {'descript

# END - Next Topic React SQL Agents